# Random Forest Models: Classification and Regression

This notebook has a similar content and structure as `Rand_Forest_ML.ipynb`, which describes how to implement Zero-Inflated Random Forest models to predict board counts at each bus stop.

 * Requiered libraries:

In [ ]:
library(randomForest)
library(mlbench)
library(caret)
library(e1071)
library(dplyr)
library(tidyr)
library(readr)
library(rFerns)
library(ranger)

In [18]:
pre_board_train = read_csv('data/jmartinez/Data_for_RF_Models/Board_Counts/route_4/direction0/bus_stop_12/pre_lock_train_data.csv')
pre_board_test = read_csv('data/jmartinez/Data_for_RF_Models/Board_Counts/route_4/direction0/bus_stop_12/pre_lock_test_data.csv')


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




In [185]:
RF_Ferns_and_Ranger <- function(rt, di, st, part){
    path = paste0('data', '/', 'jmartinez', '/', 'Data_for_RF_Models', '/', 'Board_Counts', '/',
                  paste('route', rt, sep = '_'), '/', paste('direction', di, sep = ''), '/',
                  paste('bus_stop', st, sep = '_'), '/')
    if(part == 'pre'){
        file_path_train = paste(path, 'pre_lock_train_data.csv', sep = '/')
        file_path_test = paste(path, 'pre_lock_test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    else if(part == 'post'){
        
        file_path_train = paste(path, 'post_lock_train_data.csv', sep = '/')
        file_path_test = paste(path, 'post_lock_test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    else{
        file_path_train = paste(path, 'train_data.csv', sep = '/')
        file_path_test = paste(path, 'test_data.csv', sep = '/')
        
        board_train = read_csv(file_path_train)
        board_test = read_csv(file_path_test)
        
        board_train$month = factor(board_train$month)
        board_train$service_kind = factor(board_train$service_kind)
        board_train$hour = factor(board_train$hour)

        board_test$month = factor(board_test$month)
        board_test$service_kind = factor(board_test$service_kind)
        board_test$hour = factor(board_test$hour)
    }
    
    train_month_levels = length(levels(board_train$month))
    train_service_kind_levels = length(levels(board_train$service_kind))
    train_hour_levels = length(levels(board_train$hour))    
    
    board_test = board_test %>%
        filter(hour %in% intersect(unique(board_test$hour), unique(board_train$hour)))
    
    if(train_month_levels > 1){
        if(train_service_kind_levels > 1){
            if(train_hour_levels > 1){
                board_train = board_train 
                
                board_test = board_test 
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, service_kind, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, service_kind, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
        else{
            if(train_hour_levels > 1){
                
                board_train = board_train %>%
                    summarise(board_count, hour, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, month, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
    }
    else{
        if(train_service_kind_levels > 1){
            if(train_hour_levels > 1){
                board_train = board_train %>%
                    summarise(board_count, hour, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, service_kind, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
        else{
            if(train_hour_levels > 1){
                board_train = board_train %>%
                    summarise(board_count, hour, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, hour, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
            else{
                board_train = board_train %>%
                    summarise(board_count, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
                
                board_test = board_test %>%
                    summarise(board_count, mean_temp, mean_precip, month_average_board_count, surrounding_board_count)
            }
        }
    }
    
    n_row_train = nrow(board_train)
    
    if(n_row_train < 60){
        
        return('Insufficient data for analysis!')
    }
    else{
        y_clf_train = board_train$board_count
        y_clf_train = factor(if_else(y_clf_train == 0, 0, 1))
        
        y_clf_test = board_test$board_count
        y_clf_test = factor(if_else(y_clf_test == 0, 0, 1))
        
        Board_train_clf <- data.frame(cbind(y_clf_train, board_train[, -c(1)]))
        Board_test_clf <- data.frame(cbind(y_clf_test, board_test[, -c(1)]))
        
        #---------------------------------------------------------------------------------
        # Training characteristics for model tuning:
        #---------------------------------------------------------------------------------
        
        control <- trainControl(method='repeatedcv', 
                                number=10, 
                                repeats=3,
                                search = 'random')
        
        #---------------------------------------------------------------------------------
        # Classification using Random Ferns:
        #---------------------------------------------------------------------------------
        
        set.seed(1)
        rf_random <- train(y_clf_train ~ .,
                           data = Board_train_clf,
                           method = 'rFerns',
                           metric = 'Accuracy',
                           tuneLength  = 20, 
                           trControl = control)
        
        RF_Ferns <- print(rf_random)
                
        rf_random_pred <- predict(rf_random, newdata = Board_test_clf)
        
        rf_random_conf_mat <- confusionMatrix(y_clf_test, rf_random_pred)
        rf_random_conf_mat <- data.frame(rf_random_conf_mat[4])
        colnames(rf_random_conf_mat) <- c('Value')
        # Index for regression data:
        
        index_for_reg <- which(rf_random_pred == '1', arr.ind = T)
        #---------------------------------------------------------------------------------
        # Regression Model using Ranger:
        
                
        set.seed(1)
        rf_reg_ranger <- train(board_count ~ .,
                               data = (board_train %>% filter(board_count > 0)),
                               method = 'ranger',
                               metric = 'RMSE',
                               tuneLength  = 15, 
                               trControl = control)
        
        RF_Ranger <- print(rf_reg_ranger)
                
        #----------------------------------------------------------------------------------
        # Validation:
        Board_Test_Val = board_test
        nrow_test = (1:nrow(board_test))
        
        Board_Test_Val$index = nrow_test
        Board_test_reg = Board_Test_Val[index_for_reg, ]
        
        rf_reg_ranger_pred <- predict(rf_reg_ranger, newdata = Board_test_reg)
        Board_test_reg$Ranger_Pred = rf_reg_ranger_pred
        
        Board_Test_Val = left_join(Board_Test_Val, Board_test_reg, by = 'index')
        
        Board_Test_Val = Board_Test_Val %>%
            mutate(RF_Pred = if_else(is.na(Ranger_Pred) == T, 0, Ranger_Pred))
        
        board_test$RF_Pred = Board_Test_Val$RF_Pred
                
        RF_test_RMSE = sqrt(mean((board_test$board_count - board_test$RF_Pred)^{2}))
    }
    
    if(part == 'pre'){
        file_path_clf = paste(path, 'pre_lock_RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'pre_Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'pre_lock_RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'pre_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Pre_Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Pre_Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
        
    }
    else if(part == 'post'){
        
        file_path_clf = paste(path, 'post_lock_RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'post_Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'post_lock_RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'post_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Post_Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Post_Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
    }
    else{
        file_path_clf = paste(path, 'RF_Fern.txt', sep = '/')
        file_path_clf_conf_mat = paste(path, 'Conf_Mat_RF_Fern.csv', sep = '/')
        
        file_path_reg = paste(path, 'RF_Reg.txt', sep = '/')
        file_path_RF_Chart = paste(path, 'pre_RF_Chart.csv', sep = '/')
        
        final_clf_model = paste(path, 'Random_Ferns_model.rds')
        final_reg_model = paste(path, 'Random_Forest_RANGER_model.rds')
        
        write.table(RF_Ferns, file_path_clf)
        write.csv(rf_random_conf_mat, file_path_clf_conf_mat)
        
        write.table(RF_Ranger, file_path_reg)
        write.csv(board_test, file_path_RF_Chart)
        
        saveRDS(rf_random, final_clf_model)
        saveRDS(rf_reg_ranger, final_reg_model)
    }
    
    
    return('Done!')
}

In [3]:
Pre_lock = read_csv('data/jmartinez/Transit_Data/Pre_lock.csv')


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  scheduled_arrival_time = col_datetime(format = ""),
  actual_arrival_time = col_datetime(format = ""),
  direction_desc = col_character(),
  service_period = col_character(),
  date = col_date(format = ""),
  scheduled_datetime = col_datetime(format = ""),
  actual_arrival_datetime = col_datetime(format = ""),
  trip_start_time = col_datetime(format = ""),
  trip_date = col_date(format = ""),
  service_kind = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [4]:
Post_lock = read_csv('data/jmartinez/Transit_Data/Post_lock.csv')


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  scheduled_arrival_time = col_datetime(format = ""),
  actual_arrival_time = col_datetime(format = ""),
  direction_desc = col_character(),
  service_period = col_character(),
  date = col_date(format = ""),
  scheduled_datetime = col_datetime(format = ""),
  actual_arrival_datetime = col_datetime(format = ""),
  trip_start_time = col_datetime(format = ""),
  trip_date = col_date(format = ""),
  service_kind = col_character()
)
ℹ Use `spec()` for the full column specifications.




In [5]:
Pre_lock$trip_id = as.character(Pre_lock$trip_id)
Pre_lock$stop_id = as.character(Pre_lock$stop_id)
Pre_lock$route_id = as.character(Pre_lock$route_id)
Pre_lock$direction_id = as.character(Pre_lock$direction_id)
Pre_lock$hour = factor(Pre_lock$hour)
Pre_lock$month = factor(Pre_lock$month)

In [6]:
Post_lock$trip_id = as.character(Post_lock$trip_id)
Post_lock$stop_id = as.character(Post_lock$stop_id)
Post_lock$route_id = as.character(Post_lock$route_id)
Post_lock$direction_id = as.character(Post_lock$direction_id)
Post_lock$hour = factor(Post_lock$hour)
Post_lock$month = factor(Post_lock$month)

In [25]:
bus_stops_r4_dir0_pre = Pre_lock %>%  
    filter(route_id == '4', direction_id == '0') %>%
    pull(stop_id)

bus_stops_r4_dir0_pre = unique(bus_stops_r4_dir0_pre)

In [47]:
which(bus_stops_r4_dir0_pre == '1882', arr.ind = T)

[1] 142

In [ ]:
#

In [49]:
bus_stops_r4_dir0_pre

[1] "354"  "505"  "284"  "285"  "713"  "286"  "287"  "1351" "12"   "1555"
 [11] "1579" "1354" "1353" "17"   "742"  "805"  "806"  "807"  "1475" "1474"
 [21] "811"  "812"  "813"  "814"  "815"  "816"  "817"  "818"  "819"  "820" 
 [31] "821"  "822"  "823"  "824"  "825"  "826"  "827"  "828"  "829"  "321" 
 [41] "322"  "830"  "831"  "832"  "833"  "834"  "835"  "836"  "837"  "838" 
 [51] "839"  "840"  "841"  "842"  "843"  "844"  "845"  "846"  "848"  "849" 
 [61] "850"  "851"  "852"  "854"  "892"  "855"  "1875" "856"  "894"  "784" 
 [71] "857"  "858"  "283"  "859"  "897"  "860"  "861"  "898"  "1873" "1848"
 [81] "1849" "1850" "1819" "867"  "1877" "868"  "869"  "870"  "871"  "872" 
 [91] "873"  "874"  "875"  "876"  "877"  "879"  "880"  "881"  "882"  "883" 
[101] "884"  "885"  "886"  "887"  "888"  "889"  "890"  "891"  "1485" "1876"
[111] "1482" "1465" "1466" "1468" "1469" "1470" "1471" "1472" "1473" "1878"
[121] "18"   "19"   "1557" "1357" "21"   "22"   "23"   "24"   "25"   "900" 
[131] "901"  "902"  "903"  "1908" "905"  "1909" "907"  "908"  "1910" "1880"
[141] "1917" "1882"

In [48]:
for(i in bus_stops_r4_dir0_pre[1:length(bus_stops_r4_dir0_pre)]){
    print(i)
    RF_Ferns_and_Ranger('4', '0', i, 'pre')
    print('Done!')
    print('_/|_/|_')
}

[1] NA


ERROR: Error: 'data/jmartinez/Data_for_RF_Models/Board_Counts/route_4/direction0/bus_stop_NA//pre_lock_train_data.csv' does not exist in current working directory ('/home/jmartinez').


### Post-lockdown

In [ ]:
bus_stops_r4_dir0_post = Post_lock %>%  
    filter(route_id == '4', direction_id == '0') %>%
    pull(stop_id)

bus_stops_r4_dir0_post = unique(bus_stops_r4_dir0_post)
bus_stops_r4_dir0_post

In [ ]:
which(bus_stops_r4_dir0_post == , arr.ind =T)

In [ ]:
for(i in bus_stops_r4_dir0_post[1:length(bus_stops_r4_dir0_post)]){
    print(i)
    RF_Ferns_and_Ranger('4', '0', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

In [222]:
bus_stops_r4_dir0_post = Post_lock %>%  
    filter(route_id == '4', direction_id == '1') %>%
    pull(stop_id)

bus_stops_r4_dir1_post = unique(bus_stops_r4_dir0_post)
bus_stops_r4_dir1_post

[1] "1883" "923"  "924"  "925"  "926"  "927"  "928"  "929"  "930"  "931" 
 [11] "932"  "933"  "934"  "935"  "936"  "1578" "939"  "941"  "1860" "943" 
 [21] "1477" "946"  "947"  "948"  "1919" "1820" "951"  "1889" "1853" "952" 
 [31] "953"  "954"  "955"  "956"  "957"  "959"  "960"  "961"  "962"  "963" 
 [41] "964"  "966"  "967"  "968"  "969"  "970"  "971"  "972"  "1545" "1911"
 [51] "975"  "976"  "977"  "978"  "979"  "980"  "1862" "981"  "982"  "983" 
 [61] "984"  "985"  "986"  "987"  "988"  "989"  "990"  "991"  "992"  "993" 
 [71] "994"  "995"  "407"  "408"  "409"  "412"  "413"  "996"  "997"  "998" 
 [81] "999"  "488"  "489"  "569"  "789"  "1351" "1885" "1003" "1004" "1015"
 [91] "1910" "12"   "1555" "1579" "1354" "1353" "17"   "1016" "742"  "1017"
[101] "805"  "1018" "806"  "1019" "1475" "1020" "1021" "1022" "1023" "505" 
[111] "690"  "1487" "138"  "139"  "140"  "141"  "142"  "143"  "771"  "144" 
[121] "145"  "163"  "164"  "165"  "1361" "1589" "1884" "1560" "166"  "167" 
[131] "169"  "170"  "171"  "172"  "807"  "1878" "2148" "2197" "2245" "2223"
[141] "1976" "1975" "2063" "2068" "2076" "1922" "2010" "568"  "2198" "808"

In [ ]:
# 1883, 412, 996, 1878

In [229]:
which(bus_stops_r4_dir1_post == '1878', arr.ind =T)

[1] 136

In [ ]:
for(i in bus_stops_r4_dir1_post[137:length(bus_stops_r4_dir1_post)]){
    print(i)
    RF_Ferns_and_Ranger('4', '1', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] "2148"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

108 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 98, 97, 98, 96, 97, 97, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9143434  0.8209537
   2     0.9146465  0.8233835
   3     0.9213131  0.8368290
   5     0.9083333  0.8109977
   6     0.9055556  0.8027743
   7     0.9028283  0.7973770
   9     0.8776768  0.7519671
  10     0.8870707  0.7668673
  11     0.8716162  0.7398755
  13     0.8591919  0.7192164
  14     0.8531313  0.7057861
  15     0.8473232  0.6987384

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 3.


## Route 1

### Direction 0

#### Pre-lockdown

In [7]:
bus_stops_r1_dir0_pre = Pre_lock %>%  
    filter(route_id == '1', direction_id == '0') %>%
    pull(stop_id)

In [8]:
bus_stops_r1_dir0_pre = unique(bus_stops_r1_dir0_pre)
bus_stops_r1_dir0_pre

[1] "1351" "12"   "1555" "1579" "1354" "1353" "17"   "742"  "18"   "19"  
[11] "1557" "1357" "21"   "22"   "23"   "24"   "25"   "26"   "27"   "28"  
[21] "354"  "505"  "784"  "283"  "284"  "285"  "713"  "286"  "287"  "95"  
[31] "44"

In [130]:
which(bus_stops_r1_dir0_pre == '95', arr.ind = T)

[1] 30

In [61]:
for(i in bus_stops_r1_dir0_pre[31:length(bus_stops_r1_dir0_pre)]){
    print(i)
    RF_Ferns_and_Ranger('1', '0', i, 'pre')
    print('Done!')
    print('_/|_/|_')
}

[1] "44"


ERROR: Error: 'data/jmartinez/Data_for_RF_Models/Board_Counts/route_1/direction0/bus_stop_44//pre_lock_train_data.csv' does not exist in current working directory ('/home/jmartinez').


#### Post-lockdown

In [128]:
bus_stops_r1_dir0_post = Post_lock %>%  
    filter(route_id == '1', direction_id == '0') %>%
    pull(stop_id)

bus_stops_r1_dir0_post = unique(bus_stops_r1_dir0_post)
bus_stops_r1_dir0_post

[1] "354"  "505"  "784"  "283"  "284"  "285"  "713"  "286"  "287"  "1351"
[11] "12"   "1555" "1579" "1354" "1353" "17"   "742"  "18"   "19"   "1557"
[21] "1357" "21"   "22"   "23"   "24"   "25"   "26"   "27"   "28"   "95"

In [ ]:
# 

In [ ]:
which(bus_stops_r1_dir0_post == '95', arr.ind = T)

In [129]:
for(i in bus_stops_r1_dir0_post[1:length(bus_stops_r1_dir0_post)]){
    print(i)
    RF_Ferns_and_Ranger('1', '0', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] "354"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_logical(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_logical(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"
[1] "505"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"
[1] "784"


...

Random Forest 

44 samples
 7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 40, 40, 40, 39, 40, 40, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   RMSE       Rsquared   MAE      
   1             25    extratrees  1.3644888  0.2995922  1.0050088
   1             25    variance    1.3412926  0.2561793  0.9735577
   2             14    maxstat     1.0446010  0.2105315  0.8256427
   5             10    extratrees  1.2231017  0.2943209  0.9247631
   7              9    extratrees  1.1916355  0.2772164  0.9052714
   7             20    extratrees  1.2443799  0.2782616  0.9371139
   9              3    maxstat     0.9809154  0.1812394  0.7888799
  10              1    extratrees  0.9894688  0.2754269  0.7986987
  10             12    variance    1.1741154  0.2043843  0.8860983
  11              5    extratrees  1.1292743  0.2626160  0.8773116
  14              5    variance    1.0968977  0


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

154 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 139, 138, 138, 139, 138, 139, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9890079  0.9739419
   2     0.9869246  0.9720486
   3     0.9867857  0.9696326
   5     0.9786111  0.9532681
   6     0.9760913  0.9481105
   7     0.9784524  0.9520732
   9     0.9719246  0.9387033
  10     0.9717857  0.9369070
  11     0.9717857  0.9389720
  13     0.9674802  0.9296124
  14     0.9655357  0.9256311
  15     0.9631746  0.9188419

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 1.
Random Forest 

47 samples
 7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 43, 42, 42, 42, 43, 42, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  spl


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"


In [ ]:
#### Post-lockdown 

In [131]:
bus_stops_r1_dir1_post = Post_lock %>%  
    filter(route_id == '1', direction_id == '1') %>%
    pull(stop_id)

In [132]:
bus_stops_r1_dir1_post = unique(bus_stops_r1_dir1_post)
bus_stops_r1_dir1_post

[1] "95"   "1891" "1915" "72"   "1892" "161"  "74"   "75"   "158"  "155" 
[11] "146"  "81"   "90"   "1486" "1899" "148"  "1731" "1797" "150"  "151" 
[21] "152"  "103"  "104"  "105"  "107"  "108"  "109"  "110"  "111"  "112" 
[31] "113"  "114"  "115"  "116"  "1912" "118"  "1914" "1358" "1582" "124" 
[41] "125"  "126"  "127"  "128"  "129"  "130"  "132"  "133"  "134"  "136" 
[51] "137"  "1487" "138"  "139"  "140"  "141"  "142"  "143"  "771"  "144" 
[61] "145"  "157"  "1913" "100"  "101"  "106"  "1488" "1684" "1560" "163" 
[71] "164"  "165"  "1361" "1589" "166"  "167"  "169"  "170"  "171"  "172" 
[81] "505"  "102"  "690"

In [ ]:
# c(95,72, 1892, 1582, 126, 127, 140, 144, 102, 690)

In [148]:
which(bus_stops_r1_dir1_post == '102', arr.ind = T)

[1] 82

In [149]:
for(i in bus_stops_r1_dir1_post[83:length(bus_stops_r1_dir1_post)]){
    print(i)
    RF_Ferns_and_Ranger('1', '1', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] "690"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




[1] "Done!"
[1] "_/|_/|_"


# Route 9

## Direction 0

### Pre-lockdown

In [69]:
bus_stops_r9_dir0_pre = Pre_lock %>%  
    filter(route_id == '9', direction_id == '0') %>%
    pull(stop_id)

bus_stops_r9_dir0_pre = unique(bus_stops_r9_dir0_pre)
bus_stops_r9_dir0_pre

[1] "354"  "505"  "784"  "283"  "284"  "285"  "713"  "286"  "287"  "1351"
[11] "12"   "1555" "1579" "1354" "1353" "17"   "742"  "19"   "1557" "1357"
[21] "1499" "1084" "1085" "1086" "1087" "1088" "1089" "1090" "1091" "1092"
[31] "1093" "1094" "1095" "1096" "1097" "1496" "1558" "1099" "1139" "1100"
[41] "1101" "1102" "1103" "1104" "1105" "1106" "1107" "1108" "1109" "1110"
[51] "1111" "1112" "1113" "1114" "1115" "1116" "1117" "1118" "1119" "1120"
[61] "1121" "1122" "1123" "1124" "1125" "1126" "1127" "1128" "1129" "1130"
[71] "1131" "1132" "1133" "1134" "1498" "1559" "1136" "1710" "1224"

In [ ]:
# c(354,283, 285, 713, 286, 287, 1087, 1710)

In [79]:
which(bus_stops_r9_dir0_pre == '1710', arr.ind = T)

[1] 78

In [80]:
for(i in bus_stops_r9_dir0_pre[79:length(bus_stops_r9_dir0_pre)]){
    print(i)
    RF_Ferns_and_Ranger('9', '0', i, 'pre')
    print('Done!')
    print('_/|_/|_')
}

[1] "1224"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

1311 samples
   7 predictor
   2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 1180, 1180, 1180, 1179, 1180, 1181, ... 
Resampling results across tuning parameters:

  depth  Accuracy  Kappa
   1     1         1    
   2     1         1    
   3     1         1    
   5     1         1    
   6     1         1    
   7     1         1    
   9     1         1    
  10     1         1    
  11     1         1    
  13     1         1    
  14     1         1    
  15     1         1    

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 1.
Random Forest 

508 samples
  7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 458, 457, 457, 457, 457, 457, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   RMSE       Rsquared    MAE      
   1  

### Post-lockdown

In [216]:
bus_stops_r9_dir0_post = Post_lock %>%  
    filter(route_id == '9', direction_id == '0') %>%
    pull(stop_id)

bus_stops_r9_dir0_post = unique(bus_stops_r9_dir0_post)
bus_stops_r9_dir0_post

[1] "354"  "505"  "784"  "283"  "284"  "285"  "713"  "286"  "287"  "1351"
[11] "12"   "1555" "1579" "1354" "1353" "17"   "742"  "19"   "1557" "1357"
[21] "1499" "1084" "1085" "1086" "1087" "1088" "1089" "1090" "1091" "1092"
[31] "1093" "1094" "1095" "1096" "1097" "1496" "1558" "1099" "1139" "1100"
[41] "1101" "1102" "1103" "1104" "1105" "1106" "1107" "1108" "1109" "1110"
[51] "1111" "1112" "1113" "1224" "1115" "1116" "1117" "1118" "1119" "1120"
[61] "1121" "1122" "1123" "1124" "1125" "1126" "1127" "1128" "1129" "1130"
[71] "1131" "1132" "1133" "1134" "1498" "1559" "1136" "1710"

In [217]:
# c(1087, 1088, 1089, 1096, 1104, 1110, 1122, 1129, 1559, 1710)

In [220]:
which(bus_stops_r9_dir0_post == '1710', arr.ind = T)

[1] 78

In [219]:
for(i in bus_stops_r9_dir0_post[77:length(bus_stops_r9_dir0_post)]){
    print(i)
    RF_Ferns_and_Ranger('9', '0', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] "1136"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

168 samples
  7 predictor
  2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 152, 151, 152, 150, 151, 152, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa    
   1     0.9566721  0.9023890
   2     0.9740877  0.9407916
   3     0.9685594  0.9266079
   5     0.9584014  0.9063984
   6     0.9606073  0.9145374
   7     0.9743328  0.9405420
   9     0.9623230  0.9119522
  10     0.9548339  0.8962510
  11     0.9403867  0.8644759
  13     0.9362064  0.8528141
  14     0.9326389  0.8450435
  15     0.9164760  0.8131112

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 7.


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Random Forest 

44 samples
 7 predictor

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 41, 39, 39, 40, 40, 39, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   RMSE       Rsquared   MAE      
   1             14    extratrees  1.0826619  0.2805947  0.9113604
   1             15    maxstat     0.9781982  0.2471572  0.8405644
   2              5    variance    1.0384960  0.2763110  0.8737324
   5              6    extratrees  1.0404080  0.2965105  0.8871659
   7              6    maxstat     0.9511712  0.2547429  0.8178960
   7              9    maxstat     0.9621994  0.2570325  0.8263608
   9             10    extratrees  1.0327561  0.2796636  0.8778915
  10              1    extratrees  0.9463656  0.3040321  0.8129537
  10              3    maxstat     0.9422135  0.2465485  0.8105105
  11              5    maxstat     0.9451559  0.2670973  0.8136582
  14              2    extratrees  0.9620750  0


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




ERROR: Error in na.fail.default(structure(list(y_clf_train = structure(c(1L, : missing values in object


## Direction 1

### Pre-lockdown

In [95]:
bus_stops_r9_dir1_pre = Pre_lock %>%  
    filter(route_id == '9', direction_id == '1') %>%
    pull(stop_id)

bus_stops_r9_dir1_pre = unique(bus_stops_r9_dir1_pre)
bus_stops_r9_dir1_pre

[1] "1710" "1701" "1706" "1707" "1708" "1142" "1143" "1144" "1145" "1146"
[11] "1147" "1501" "1148" "1149" "1150" "1151" "1152" "1153" "1154" "1155"
[21] "1156" "1157" "1502" "1159" "1161" "1162" "1163" "1164" "1548" "1503"
[31] "1556" "1168" "1169" "1171" "1172" "1173" "1174" "1175" "1504" "1177"
[41] "1178" "1179" "1180" "1181" "1505" "1183" "1184" "1185" "1186" "1187"
[51] "1188" "1189" "1190" "1191" "1192" "1193" "1461" "142"  "143"  "771" 
[61] "144"  "145"  "1560" "163"  "164"  "165"  "1361" "1589" "166"  "167" 
[71] "169"  "170"  "171"  "172"  "505"  "690"

In [ ]:
# c(169, 170, 171, 172, 505, 690)

In [101]:
which(bus_stops_r9_dir1_pre == '171', arr.ind = T)

[1] 73

In [106]:
for(i in bus_stops_r9_dir1_pre[76:length(bus_stops_r9_dir1_pre)]){
    print(i)
    RF_Ferns_and_Ranger('9', '1', i, 'pre')
    print('Done!')
    print('_/|_/|_')
}

[1] "690"



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_logical(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




ERROR: Error in na.fail.default(structure(list(y_clf_train = structure(c(1L, : missing values in object


### Post-lockdown

In [172]:
bus_stops_r9_dir1_post = Post_lock %>%  
    filter(route_id == '9', direction_id == '1') %>%
    pull(stop_id)

bus_stops_r9_dir1_post = unique(bus_stops_r9_dir1_post)
bus_stops_r9_dir1_post

[1] "1710" "1701" "1706" "1707" "1708" "1142" "1143" "1144" "1145" "1146"
[11] "1147" "1501" "1148" "1149" "1150" "1151" "1152" "1153" "1154" "1155"
[21] "1156" "1157" "1502" "1159" "1161" "1162" "1163" "1164" "1548" "1503"
[31] "1556" "1168" "1169" "1171" "1172" "1173" "1174" "1175" "1504" "1177"
[41] "1178" "1179" "1180" "1181" "1505" "1183" "1184" "1185" "1186" "1187"
[51] "1188" "1189" "1190" "1191" "1192" "1193" "1461" "142"  "143"  "771" 
[61] "144"  "145"  "1560" "163"  "164"  "165"  "1361" "1589" "166"  "167" 
[71] "169"  "170"  "171"  "172"  "505"  "690"

In [ ]:
# c(1710, 1143, 1156, 1157, 1189, 690, 1155, 1156, 1157, 1189, 1159, 690)

In [182]:
which(bus_stops_r9_dir1_post == '690', arr.ind = T)

[1] 76

In [183]:
for(i in bus_stops_r9_dir1_post[77:length(bus_stops_r9_dir1_post)]){
    print(i)
    RF_Ferns_and_Ranger('9', '1', i, 'post')
    print('Done!')
    print('_/|_/|_')
}

[1] NA


ERROR: Error: 'data/jmartinez/Data_for_RF_Models/Board_Counts/route_9/direction1/bus_stop_NA//post_lock_train_data.csv' does not exist in current working directory ('/home/jmartinez').


In [192]:
RF_Ferns_and_Ranger('9', '0', '1091', 'post')


── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  month = col_double(),
  service_kind = col_character(),
  hour = col_double(),
  board_count = col_double(),
  mean_temp = col_double(),
  mean_precip = col_double(),
  month_average_board_count = col_double(),
  surrounding_board_count = col_double()
)




Random Ferns 

88 samples
 7 predictor
 2 classes: '0', '1' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 79, 79, 80, 78, 80, 80, ... 
Resampling results across tuning parameters:

  depth  Accuracy   Kappa     
   1     0.6129630  0.16971491
   2     0.6264815  0.18166878
   3     0.6070370  0.16449112
   5     0.5797222  0.15321028
   6     0.5512963  0.12307301
   7     0.5220370  0.10031851
   9     0.4419444  0.07596244
  10     0.4262037  0.06873596
  11     0.3895370  0.05568498
  13     0.3087963  0.03789853
  14     0.2811111  0.03368468
  15     0.2550926  0.02701383

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was depth = 2.


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Random Forest 

6 samples
7 predictors

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 5, 5, 5, 5, 5, 5, ... 
Resampling results across tuning parameters:

  min.node.size  mtry  splitrule   RMSE       Rsquared  MAE      
  1               5    maxstat     0.8538463  NaN       0.8538463
  1               7    maxstat     0.8623630  NaN       0.8623630
  2               9    extratrees  0.8769370  NaN       0.8769370
  2               9    maxstat     0.8634241  NaN       0.8634241
  2              12    variance    0.8102259  NaN       0.8102259
  2              20    extratrees  0.8614556  NaN       0.8614556
  3               5    variance    0.8206037  NaN       0.8206037
  3              14    extratrees  0.8592370  NaN       0.8592370
  3              21    extratrees  0.8695667  NaN       0.8695667
  5               2    extratrees  0.8065333  NaN       0.8065333
  5              10    variance    0.8003556  NaN       0.8003556

[1] "Done!"